In [ ]:
from utils import *
from keras.models import *
from keras.layers import *
import tensorflow as tf
from keras.optimizers import *
import keras.backend as K
from keras.utils import plot_model
from random import shuffle

In [ ]:
nepochs=5
lr=1e-4

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    vertical_flip=True,
                    brightness_range=[0.2,1.8],
                    fill_mode='nearest')

image_mask_paths = [("../data/masks/image_%d.tif"%i,"../data/masks/image_%d_mask.png"%i) for i in range(1,19)]

test_paths = image_mask_paths[:int(2)]
train_paths = image_mask_paths[int(2):int(5)]

In [ ]:
input_size = (256,256,1)

inputs = Input(input_size)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
merge6 = concatenate([drop4,up6], axis = 3)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = concatenate([conv3,up7], axis = 3)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 = concatenate([conv2,up8], axis = 3)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = concatenate([conv1,up9], axis = 3)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

model = Model(input = inputs, output = conv10)

In [ ]:
#model = unet(input_size=input_size+(1,))
optimizer = RMSprop
model.compile(optimizer = optimizer(lr = lr), loss = 'binary_crossentropy', metrics = ['accuracy'])
#plot_model(model, to_file='model.png',show_shapes=True)

In [ ]:
img_list = []
mask_list = []
for epoch in range(nepochs-1):
    shuffle(train_paths)
    out_imgs = np.zeros((16,)+(256,256)+(1,))
    out_masks = np.zeros((16,)+(256,256)+(1,))
    for i, img_mask_path in enumerate(train_paths):
        img, mask = read_data(img_mask_path)
        #print(img.shape, mask.shape)
        out_imgs[i,...] = img
        out_masks[i,...] = mask
        #img_list.append(img)
        #mask_list.append(mask)
        
    loss = model.train_on_batch(out_imgs,out_masks)
    #print(type(loss))

In [ ]:
test_loss = []

out_imgs = np.zeros((16,)+(256,256)+(1,))
out_masks = np.zeros((16,)+(256,256)+(1,))
for i, img_mask_path in enumerate(test_paths):
    img, mask = read_data(img_mask_path)
    #print(img.shape, mask.shape)
    out_imgs[i,...] = img
    out_masks[i,...] = mask
    #img_list.append(img)
    #mask_list.append(mask)
    
prediction = model.predict_on_batch(out_imgs)
if epoch % 20 == 0 or epoch == nepochs-1:
    save_output(out_imgs[0,...],out_masks[0,...],prediction[0,...],index=i,epoch=epoch)
test_loss.append(model.test_on_batch(out_imgs,out_masks))
    
test_loss = np.mean(np.array(test_loss),axis=0)
save_model_unet(model,epoch,test_loss[1])